In [76]:
#import necessary packages
import pandas as pd
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.sparse as sp
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from collections import defaultdict

import torch
import math
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pyvis.network import Network
import time

# Data Preprocessing

In [ ]:
dataset_location = 'synthea_1m_fhir_3_0_May_24/output_1/fhir/'
directory = os.fsencode(dataset_location)

In [ ]:
#convert fhir json to pandas dataframe
encounters,conditions,patient_ids,procedures, observations, dignotstics_reports,\
immunizations, medication_orders, careplans = ([] for _ in range(9))
encounters_display,conditions_display,patient_ids_display,procedures_display, observations_display, dignotstics_reports_display,\
immunizations_display, medication_orders_display, careplans_display = ([] for _ in range(9))

for i, file in enumerate(os.listdir(directory)):
    filename = os.fsdecode(file)
    with open(dataset_location+filename, "r") as read_file:
        data = json.load(read_file)
        resources = data['entry']
        patient_id = resources[0]['resource']['id']
        name = resources[0]['resource']['name'][0]['family']
        patient_ids.append(patient_id)
        patient_ids_display.append(name)
        patient_encounters,patient_conditions, patient_procedures,patient_observations,\
        patient_diagnostics_reports, patient_immunizations, patient_medication_orders,\
        patient_careplans = (set() for _ in range(8))
        patient_encounters_display,patient_conditions_display, patient_procedures_display,patient_observations_display,\
        patient_diagnostics_reports_display, patient_immunizations_display, patient_medication_orders_display,\
        patient_careplans_display = (set() for _ in range(8))
        
   
        for resource in resources:
            resource_type = resource['resource']['resourceType']
            if resource_type =='Encounter':
                patient_encounters.add(resource['resource']['type'][0]['coding'][0]['code'])
                patient_encounters_display.add(resource['resource']['type'][0].get('text',''))
            elif resource_type =='Condition':
                patient_conditions.add(resource['resource']['code']['coding'][0]['code'])
                patient_conditions_display.add(resource['resource']['code']['coding'][0]['display'])
            elif resource_type =='Procedure':
                code = resource['resource']['code']['coding'][0]['code']
                display = resource['resource']['code']['coding'][0]['display']
                #ignore documentation of current medications
                if code!='428191000124101':
                    patient_procedures.add(code)  
                    patient_procedures_display.add(display) 
            elif resource_type =='Observation':
                patient_observations.add(resource['resource']['code']['coding'][0]['code'])
                patient_observations_display.add(resource['resource']['code']['coding'][0]['display'])
            elif resource_type =='DiagnosticReport':
                patient_diagnostics_reports.add(resource['resource']['code']['coding'][0]['code'])
                patient_diagnostics_reports_display.add(resource['resource']['code']['coding'][0]['display'])
            elif resource_type == 'Immunization':
                patient_immunizations.add(resource['resource']['vaccineCode']['coding'][0]['code'])
                patient_immunizations_display.add(resource['resource']['vaccineCode']['coding'][0]['display'])
            elif resource_type == medication:
                patient_medication_orders.add(resource['resource']['code']['coding'][0]['code'])
                patient_medication_orders_display.add(resource['resource']['code']['coding'][0]['display'])
            elif resource_type == 'CarePlan':
                patient_careplans.add(resource['resource']['category'][0]['coding'][0]['code'])
                patient_careplans_display.add(resource['resource']['category'][0]['coding'][0]['display'])
                
        encounters.append(','.join(patient_encounters))
        conditions.append(','.join(patient_conditions))
        procedures.append(','.join(patient_procedures))
        observations.append(','.join(patient_observations))
        dignotstics_reports.append(','.join(patient_diagnostics_reports))
        immunizations.append(','.join(patient_immunizations))
        medication_orders.append(','.join(patient_medication_orders))
        careplans.append(','.join(patient_careplans))
        
        encounters_display.append(','.join(patient_encounters_display))
        conditions_display.append(','.join(patient_conditions_display))
        procedures_display.append(','.join(patient_procedures_display))
        observations_display.append(','.join(patient_observations_display))
        dignotstics_reports_display.append(','.join(patient_diagnostics_reports_display))
        immunizations_display.append(','.join(patient_immunizations_display))
        medication_orders_display.append(','.join(patient_medication_orders_display))
        careplans_display.append(','.join(patient_careplans_display))
#     if i==10:
#         break
        
        
data = {'patient': patient_ids,'Encounter': encounters, 'Condition': conditions, 'Procedure':procedures, 'Observation':observations, 
        'Diagnostics Report':dignotstics_reports, 'Immunization':immunizations, 'Medication Orders':medication_orders, 'Careplan': careplans,
       'patient display': patient_ids_display,'Encounter display': encounters_display, 'Condition display': conditions_display, 'Procedure display':procedures_display, 'Observation display':observations_display, 
        'Diagnostics Report display':dignotstics_reports_display, 'Immunization display':immunizations_display, 'Medication Orders display':medication_orders_display, 'Careplan display': careplans_display}
resource_df = pd.DataFrame(data)

In [ ]:
resource_df.replace('', np.nan, inplace=True)
resource_df.dropna(axis=0, how='any', subset=['Condition','Procedure', 'Medication Orders', 'Observation'], inplace=True)
resource_df.isnull().sum()

In [ ]:
condition_df = resource_df[['patient','patient display','Procedure','Procedure display',
                            'Medication Orders','Medication Orders display','Observation', 
                            'Observation display','Condition','Condition display']]

In [ ]:
condition_df['Document'] = condition_df['Procedure'].apply(lambda x : ' '.join(x.split(',')))+ ' ' + condition_df['Medication Orders'].apply(lambda x : ' '.join(x.split(',')))

In [ ]:
conditions = [cond for condition in condition_df['Condition'] for cond in condition.split(',') ]
conditions_count = dict(sorted(Counter(conditions).items(), key=lambda item:item[1]))
topic_candidates = list(conditions_count.keys())[-5:]

In [ ]:
conditions_bundle = condition_df.Condition.tolist()
common = []
for case in conditions_bundle:
    common.append(np.in1d(np.array(topic_candidates), np.array(case)).sum())
unique_indicies = [index for index, count in enumerate(common) if count==1]
selected_df = condition_df.iloc[unique_indicies]

In [ ]:
selected_df['Condition']=selected_df['Condition']\
.map(lambda x: list(set(x.split(',')) & set(topic_candidates))[0])

In [ ]:
conditions = [cond for condition in selected_df['Condition'] for cond in condition.split(',') ]
conditions_count = dict(sorted(Counter(conditions).items(), key=lambda item:item[1]))
conditions_count

In [ ]:
selected_df.to_csv('data/synthea.csv', index=False)

# Build the Text-GCN graph

In [ ]:
synthea_df = pd.read_csv('data/synthea.csv')
# conditions_df = pd.read_csv('../text_gcn_on_fhir/fhirlarge.csv')
# synthea_df = conditions_df
synthea_df['Condition'].unique()

In [13]:
#train_test_split
size = len(synthea_df)
# size = len(ngtext)
train_size = int(0.75*size)
val_size = int(0.1*size)
test_size = size - train_size - val_size
print(size, train_size, test_size, val_size)

724 543 109 72


In [14]:
#shuffling
shuffled_id = np.arange(size)
random.shuffle(shuffled_id)
shuffled_document = synthea_df['Document'][shuffled_id]

In [15]:
labels = synthea_df['Condition'].values[shuffled_id]
labels_list = list(synthea_df['Condition'].unique())

In [16]:
# build vocab
word_freq = defaultdict(lambda : 0)
word_set = set()
for doc_words in shuffled_document:
    words = doc_words.split()
    for word in words:
        word_set.add(word)
        word_freq[word] += 1

vocab = list(word_set)
vocab_size = len(vocab)
word_embeddings_dim = vocab_size

In [17]:
word_doc_list = defaultdict(lambda : [])
for index, doc in enumerate(shuffled_document):
    appeared = set()
    doc_words = doc.split()
    for word in doc_words:
        if word in appeared:
            continue
        word_doc_list[word].append(index)
        appeared.add(word)
        
word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)

word_id_map = {}
for index, word in enumerate(vocab):
    word_id_map[word] = index

In [18]:
#ecoding using tfidf
tfidf_vec = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vec.fit_transform(vocab)
tfidf_matrix_array = tfidf_matrix.toarray()

### Feature matrix preparation

In [19]:
word_vector_map = {}
for word, idd in word_id_map.items():
    word_vector_map[word] = tfidf_matrix_array[idd]

In [20]:
#train data feature
row_x = []
col_x = []
data_x = []

for i, doc in enumerate(shuffled_document[:train_size+val_size]):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = doc.split()
    doc_len = len(words)
    for word in words:
#         print(word)
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector) 
            
    data_x.append(doc_vec)
x = np.array(data_x)/doc_len        


In [21]:
#train data labels
y = []
for label in labels[:train_size+val_size]:
    one_hot = [0 for _ in range(len(labels_list))]
    index = labels_list.index(label)
    one_hot[index]= 1
    y.append(one_hot)
    
y = np.array(y)

In [22]:
len(labels)

724

In [23]:
word_vectors = np.random.uniform(-0.01, 0.01,
                                 (vocab_size, word_embeddings_dim))

for i in range(len(vocab)):
    word = vocab[i]
    if word in word_vector_map:
        vector = word_vector_map[word]
        word_vectors[i] = vector

In [24]:
#test data feature
row_tx = []
col_tx = []
data_tx = []


for i, doc in enumerate(shuffled_document[-test_size:]):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = doc.split()
    doc_len = len(words)
    for word in doc_words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)
    data_tx.append(doc_vec)
tx = np.array(data_tx)/doc_len


#test data label
ty = [labels_list.index(label) for label in labels[-test_size:]]

print(tx.shape, len(ty))

(109, 53) 109


In [25]:
# allx: the the feature vectors of both labeled and unlabeled training instances
# (a superset of x)
# unlabeled training instances -> words

data_allx = []

for i, doc in enumerate(shuffled_document[:(train_size+val_size)]):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = doc.split()
    doc_len = len(words)
    for word in doc_words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)
    data_allx.append(doc_vec)
data_allx = np.array(data_allx)/doc_len
allx = np.vstack([data_allx, np.array(word_vectors)])           

ally = [labels_list.index(label) for label in labels[:train_size+val_size]]
ally.extend([-1 for _ in range(vocab_size)])

# ally = np.array(ally)

print(tx.shape, len(ty), allx.shape, len(ally), train_size+val_size+vocab_size)

(109, 53) 109 (668, 53) 668 668


### Doc word heterogeneous graph

In [60]:
# word co-occurence with context windows
window_size = 3
windows = []

for doc_words in shuffled_document:
    words = doc_words.split()
    length = len(words)
    if length <= window_size:
        windows.append(words)
    else:
        # print(length, length - window_size + 1)
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)
            
word_window_freq = defaultdict(lambda :0)
for window in windows:
    appeared = set()
    for word in window:
        if word in appeared:
            continue
        word_window_freq[word]+=1
        appeared.add(word)

word_pair_count = defaultdict(lambda :0)
for window in windows:
    for i in range(1, len(window)):
        for j in range(0,i):
            word_i = window[i]
            word_j = window[j]
            word_i_id  = word_id_map[word_i]
            word_j_id =  word_id_map[word_j]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id)
            word_pair_count[word_pair_str]+=1
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            word_pair_count[word_pair_str]+=1
            
            
# Word-Word node pmi as weights
row = []
col = []
weight = []
num_window = len(windows)

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[vocab[i]]
    word_freq_j = word_window_freq[vocab[j]]
    pmi = np.log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    row.append(train_size+val_size +i)
    col.append(train_size+val_size + j)
    weight.append(pmi)
    
    
# doc word frequency
doc_word_freq = defaultdict(lambda : 0)

for doc_id, doc in enumerate(shuffled_document):
    words = doc_words.split()
    for word in words:
        word_id = word_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        doc_word_freq[doc_word_str]+=1
        
#Word-Doc node train weights
for i, doc_words in enumerate(shuffled_document):
    words = doc_words.split()
    doc_word_set = set()
    for word in words:
        if word in doc_word_set:
            continue
        j = word_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        if i < train_size+val_size:
            row.append(i)
        else:
            row.append(i + vocab_size)
        col.append(train_size+val_size+ j)
        idf = np.log(1.0 * len(shuffled_document) /
                  word_doc_freq[vocab[j]])
        weight.append(freq * idf)
        doc_word_set.add(word)

node_size = train_size + val_size + vocab_size + test_size

adj = sp.csr_matrix(
    (weight, (row, col)), shape=(node_size, node_size))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)


print(len(row),len(col), len(weight), max(row), node_size)


2307 2307 2307 776 777


# Display the word doc hetergenous graph

In [62]:
code_display_map = {}
for code, display in zip(synthea_df['Procedure'],synthea_df['Procedure display']):
    codes = code.split(',')
    displays = display.split(',')
    for c, d in zip(codes, displays):
        code_display_map[c]=(d, 'procedure')
        
for code, display in zip(synthea_df['Medication Orders'],synthea_df['Medication Orders display']):
    codes = code.split(',')
    displays = display.split(',')
    for c, d in zip(codes, displays):
        code_display_map[c]=(d, 'medication')
        
for code, display in zip(synthea_df['Condition'],synthea_df['Condition display']):
    code_display_map[code]=(display, 'condition')

patient_condition = synthea_df[['patient display', 'Condition display']].values[shuffled_id]

In [63]:
def idtoname(row, col, adj):
    if row < train_size+val_size:
        row_id = row
        src = patient_condition[row_id][0]
        node_type_src = patient_condition[row_id][1]
    elif row >= train_size+val_size and row <train_size+val_size+vocab_size :
        row_id = row-(train_size+val_size)
        src = code_display_map[vocab[row_id]][0]
        node_type_src = code_display_map[vocab[row_id]][1]
    else:
        row_id = row-vocab_size
        src = patient_condition[row_id][0]
        node_type_src = patient_condition[row_id][1]
    col_id = col-(train_size+val_size)
    dst = code_display_map[vocab[col_id]][0]
    node_type_dst = code_display_map[vocab[col_id]][1]
    weight = adj[row][col].item()
    
    return src, dst, weight, (node_type_src, node_type_dst)

In [64]:
np.unique(patient_condition[:, 1])

array(['Acute bronchitis (disorder)', 'Otitis media', 'Prediabetes',
       'Viral sinusitis (disorder)'], dtype=object)

In [69]:
got_net = Network(height='750px', width='100%', bgcolor='#222222', font_color='white', notebook=True)
got_net.barnes_hut()
# got_net.toggle_physics(False)

In [74]:
adj_tensor = sparse_mx_to_torch_sparse_tensor(adj)
for r,c in zip(row, col):
    src, dst, weight, node_type = idtoname(r, c, adj_tensor)
    group = {'Acute bronchitis (disorder)':1,'Otitis media':2,'Prediabetes':3, 'Viral sinusitis (disorder)':4, 'procedure':5, 'medication':6, 'condition':7}
    got_net.add_node(r, label=src, title=node_type[0], group=group[node_type[0]])
    got_net.add_node(c, label=dst, title=node_type[1], group=group[node_type[1]])
    got_net.add_edge(r, c, value=weight)
    

In [80]:
got_net.show('imgs/fhir.html')

## Train on Graph Convolution Network (GCN)

In [51]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [39]:
labels = ally + ty

features_merge = np.vstack([allx, tx]).astype('float32')
features = sp.csr_matrix(features_merge, dtype=np.float32)
adj = sp.csr_matrix(
    (weight, (row, col)), shape=(node_size, node_size))

# build symmetric adjacency matrix
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

features = normalize(features)
adj = normalize(adj + sp.eye(adj.shape[0]))

In [40]:
perm = np.arange(0, node_size)
#np.random.shuffle(perm)
idx_train = perm[:train_size]
idx_val = perm[train_size:train_size + val_size]
idx_test= perm[train_size + val_size + vocab_size : ]

In [41]:
features = torch.FloatTensor(np.array(features.todense()))
labels = torch.LongTensor(labels)
adj = sparse_mx_to_torch_sparse_tensor(adj)

idx_train = torch.LongTensor(idx_train)
idx_val = torch.LongTensor(idx_val)
idx_test = torch.LongTensor(idx_test)

In [42]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [43]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [44]:
cuda = False

np.random.seed(42)
torch.manual_seed(42)
if cuda:
    torch.cuda.manual_seed(42)


In [45]:
lr = 0.01
weight_decay = 5e-4
dropout = 0.5
hidden_size =16
fastmode = False
epochs = 200

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=hidden_size,
            nclass=labels.max().item() + 1,
            dropout=dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

In [46]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [47]:
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 loss_train: 1.5055 acc_train: 0.2081 loss_val: 1.4048 acc_val: 0.3333 time: 0.0667s
Epoch: 0002 loss_train: 1.4850 acc_train: 0.2136 loss_val: 1.3734 acc_val: 0.3333 time: 0.0025s
Epoch: 0003 loss_train: 1.4600 acc_train: 0.1934 loss_val: 1.3430 acc_val: 0.3333 time: 0.0022s
Epoch: 0004 loss_train: 1.3761 acc_train: 0.2468 loss_val: 1.3133 acc_val: 0.3333 time: 0.0018s
Epoch: 0005 loss_train: 1.3689 acc_train: 0.2652 loss_val: 1.2850 acc_val: 0.3333 time: 0.0019s
Epoch: 0006 loss_train: 1.3332 acc_train: 0.3020 loss_val: 1.2574 acc_val: 0.3611 time: 0.0019s
Epoch: 0007 loss_train: 1.2937 acc_train: 0.3683 loss_val: 1.2304 acc_val: 0.6667 time: 0.0017s
Epoch: 0008 loss_train: 1.2712 acc_train: 0.4162 loss_val: 1.2046 acc_val: 0.6944 time: 0.0018s
Epoch: 0009 loss_train: 1.2277 acc_train: 0.4512 loss_val: 1.1802 acc_val: 0.6806 time: 0.0019s
Epoch: 0010 loss_train: 1.1964 acc_train: 0.5064 loss_val: 1.1568 acc_val: 0.5278 time: 0.0018s
Epoch: 0011 loss_train: 1.1864 acc_train

Epoch: 0110 loss_train: 0.2084 acc_train: 0.9263 loss_val: 0.0916 acc_val: 0.9722 time: 0.0022s
Epoch: 0111 loss_train: 0.2128 acc_train: 0.9337 loss_val: 0.0909 acc_val: 0.9722 time: 0.0019s
Epoch: 0112 loss_train: 0.2451 acc_train: 0.9337 loss_val: 0.0907 acc_val: 0.9722 time: 0.0018s
Epoch: 0113 loss_train: 0.1746 acc_train: 0.9576 loss_val: 0.0905 acc_val: 0.9722 time: 0.0017s
Epoch: 0114 loss_train: 0.1776 acc_train: 0.9558 loss_val: 0.0903 acc_val: 0.9722 time: 0.0016s
Epoch: 0115 loss_train: 0.1812 acc_train: 0.9466 loss_val: 0.0898 acc_val: 0.9722 time: 0.0017s
Epoch: 0116 loss_train: 0.1626 acc_train: 0.9374 loss_val: 0.0894 acc_val: 0.9722 time: 0.0020s
Epoch: 0117 loss_train: 0.1727 acc_train: 0.9540 loss_val: 0.0889 acc_val: 0.9722 time: 0.0020s
Epoch: 0118 loss_train: 0.1863 acc_train: 0.9392 loss_val: 0.0885 acc_val: 0.9722 time: 0.0022s
Epoch: 0119 loss_train: 0.1936 acc_train: 0.9282 loss_val: 0.0882 acc_val: 0.9722 time: 0.0019s
Epoch: 0120 loss_train: 0.1891 acc_train

In [511]:
test()

Test set results: loss= 0.0795 accuracy= 0.9633
